In [2]:
import pandas as pd

C:\Users\Horacy\AppData\Local\Temp\ipykernel_12444\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [11]:
from models.raw.bert import process_message as bert_process_message
from models.raw.openai import process_message as chatgpt_process_message
from models.raw.spacy import process_message as spacy_process_message

In [6]:
df = pd.read_csv('./data/in.csv', header=None)
print(df)

                                                    0
0   The messages will be unwrapped by sculptor Ric...
1   A film based on the internet musings of the "B...
2   Bridget Jones: The Edge of Reason was named Ev...
3   "Journalists are the lifeblood of any newspape...
4   In addition, if a flight is cancelled or delay...
..                                                ...
95  Film star Robert De Niro has returned to the t...
96  The Last Samurai, starring Tom Cruise, was the...
97  "When I was 15 years old I asked my mom if it ...
98  A US musicologist has recreated a lost musical...
99  India has ordered the deportation of Iranian-b...

[100 rows x 1 columns]


In [12]:
predictions = {
    "bert": [],
    "chatgpt": [],
    "spacy": [],
}

In [13]:
for index, row in df.iterrows():
    message = row[0]
    predictions['bert'].append(bert_process_message(message))
    predictions['chatgpt'].append(chatgpt_process_message(message))
    predictions['spacy'].append(spacy_process_message(message))

In [16]:
df2 = pd.read_csv('./data/out.csv', header=None)
print(df2)

                                                    0
0                                   Richard Wentworth
1    Theo Van Gogh, Salam Pax, Oday Rasheed, Van Gogh
2   Bridget Jones, Vera Drake, Leigh, Staunton, Os...
3                Murdoch MacLennan, Barry Fitzpatrick
4                                 Clarke, Andy Clarke
..                                                ...
95  Robert De Niro, De Niro, Clint Eastwood, Howar...
96                                         Tom Cruise
97                               Reeves, Keanu Reeves
98  Johann Sebastian Bach, Joshua Rifkin, Bach, Ba...
99               Negar Khan, Bipin Bihari, Khan, Khan

[100 rows x 1 columns]


In [29]:
metrics = {
    "bert": {
        "ground_truth": [],
        "correct_predictions": [],
        "accuracy": [],
        "precision": [],
        "recall": [],
        "f1_score": [],
    },
    "chatgpt": {
        "ground_truth": [],
        "correct_predictions": [],
        "accuracy": [],
        "precision": [],
        "recall": [],
        "f1_score": [],
    },
    "spacy": {
        "ground_truth": [],
        "correct_predictions": [],
        "accuracy": [],
        "precision": [],
        "recall": [],
        "f1_score": [],
    }
}

for index, row in df2.iterrows():
    ground = row[0].split(', ')

    metrics["bert"]["ground_truth"].append(ground)
    metrics["chatgpt"]["ground_truth"].append(ground)
    metrics["spacy"]["ground_truth"].append(ground)
    
    correct_predictions = {
        "bert": set(ground).intersection(set(predictions["bert"][index])),
        "chatgpt": set(ground).intersection(set(predictions["chatgpt"][index])),
        "spacy": set(ground).intersection(set(predictions["spacy"][index])),
    }

    metrics["bert"]["correct_predictions"].append(correct_predictions["bert"])
    metrics["chatgpt"]["correct_predictions"].append(correct_predictions["chatgpt"])
    metrics["spacy"]["correct_predictions"].append(correct_predictions["spacy"])

    accuracy = {
        "bert": len(correct_predictions["bert"]) / len(predictions["bert"]),
        "chatgpt": len(correct_predictions["chatgpt"]) / len(predictions["chatgpt"]),
        "spacy": len(correct_predictions["spacy"]) / len(predictions["spacy"]),
    }

    metrics["bert"]["accuracy"].append(accuracy["bert"])
    metrics["chatgpt"]["accuracy"].append(accuracy["chatgpt"])
    metrics["spacy"]["accuracy"].append(accuracy["spacy"])

    true_positives = {
        "bert": correct_predictions["bert"],
        "chatgpt": correct_predictions["chatgpt"],
        "spacy": correct_predictions["spacy"],
    }

    false_positives = {
        "bert": set(predictions["bert"][index]) - set(ground),
        "chatgpt": set(predictions["chatgpt"][index]) - set(ground),
        "spacy": set(predictions["spacy"][index]) - set(ground),
    }

    prec_bert_div = len(true_positives["bert"]) + len(false_positives["bert"])
    prec_chatgpt_div = len(true_positives["chatgpt"]) + len(false_positives["chatgpt"])
    prec_spacy_div = len(true_positives["spacy"]) + len(false_positives["spacy"])

    precision = {
        "bert": len(true_positives["bert"]) / (prec_bert_div if prec_bert_div != 0 else 1),
        "chatgpt": len(true_positives["chatgpt"]) / (prec_chatgpt_div if prec_chatgpt_div != 0 else 1),
        "spacy": len(true_positives["spacy"]) / (prec_spacy_div if prec_spacy_div != 0 else 1),
    }

    metrics["bert"]["precision"].append(precision["bert"])
    metrics["chatgpt"]["precision"].append(precision["chatgpt"])
    metrics["spacy"]["precision"].append(precision["spacy"])

    false_negative = {
        "bert": set(ground) - set(predictions["bert"][index]),
        "chatgpt": set(ground) - set(predictions["chatgpt"][index]),
        "spacy": set(ground) - set(predictions["spacy"][index]),
    }

    recall = {
        "bert": len(true_positives["bert"]) / (len(true_positives["bert"]) + len(false_negative["bert"])),
        "chatgpt": len(true_positives["chatgpt"]) / (len(true_positives["chatgpt"]) + len(false_negative["chatgpt"])),
        "spacy": len(true_positives["spacy"]) / (len(true_positives["spacy"]) + len(false_negative["spacy"])),
    }

    metrics["bert"]["recall"].append(recall["bert"])
    metrics["chatgpt"]["recall"].append(recall["chatgpt"])
    metrics["spacy"]["recall"].append(recall["spacy"])

    f1_bert_div = precision["bert"] + recall["bert"]
    f1_chatgpt_div = precision["chatgpt"] + recall["chatgpt"]
    f1_spacy_div = precision["spacy"] + recall["spacy"]
    
    f1_score = {
        "bert": 2 * (precision["bert"] * recall["bert"]) / (f1_bert_div if f1_bert_div != 0 else 1),
        "chatgpt": 2 * (precision["chatgpt"] * recall["chatgpt"]) / (f1_chatgpt_div if f1_chatgpt_div != 0 else 1),
        "spacy": 2 * (precision["spacy"] * recall["spacy"]) / (f1_spacy_div if f1_spacy_div != 0 else 1),
    }

    metrics["bert"]["f1_score"].append(f1_score["bert"])
    metrics["chatgpt"]["f1_score"].append(f1_score["chatgpt"])
    metrics["spacy"]["f1_score"].append(f1_score["spacy"])

In [32]:
results = {
    "bert": {
        "accuracy": sum(metrics["bert"]["accuracy"]) / len(metrics["bert"]["accuracy"]),
        "precision": sum(metrics["bert"]["precision"]) / len(metrics["bert"]["precision"]),
        "recall": sum(metrics["bert"]["recall"]) / len(metrics["bert"]["recall"]),
        "f1_score": sum(metrics["bert"]["f1_score"]) / len(metrics["bert"]["f1_score"]),
    },
    "chatgpt": {
        "accuracy": sum(metrics["chatgpt"]["accuracy"]) / len(metrics["chatgpt"]["accuracy"]),
        "precision": sum(metrics["chatgpt"]["precision"]) / len(metrics["chatgpt"]["precision"]),
        "recall": sum(metrics["chatgpt"]["recall"]) / len(metrics["chatgpt"]["recall"]),
        "f1_score": sum(metrics["chatgpt"]["f1_score"]) / len(metrics["chatgpt"]["f1_score"]),
    },
    "spacy": {
        "accuracy": sum(metrics["spacy"]["accuracy"]) / len(metrics["spacy"]["accuracy"]),
        "precision": sum(metrics["spacy"]["precision"]) / len(metrics["spacy"]["precision"]),
        "recall": sum(metrics["spacy"]["recall"]) / len(metrics["spacy"]["recall"]),
        "f1_score": sum(metrics["spacy"]["f1_score"]) / len(metrics["spacy"]["f1_score"]),
    },
}

In [34]:
print(results["bert"])
print(results["chatgpt"])
print(results["spacy"])

{'accuracy': 0.0069000000000000025, 'precision': 0.12336671661671657, 'recall': 0.12701356976356976, 'f1_score': 0.12291174530623733}
{'accuracy': 0.036699999999999955, 'precision': 0.7918882053618894, 'recall': 0.699369612843297, 'f1_score': 0.7196617554272349}
{'accuracy': 0.036499999999999984, 'precision': 0.7498587580719935, 'recall': 0.6895277704751389, 'f1_score': 0.7019123740896183}


In [35]:
metrics_df = pd.DataFrame.from_dict(metrics)
results_df = pd.DataFrame.from_dict(results)

In [38]:
metrics_df.to_csv('./data/metrics.csv', index=False, header=True)
results_df.to_csv('./data/results.csv', index=False, header=True)